# Coffee Shop Loyalty System with Redis

This notebook demonstrates how to build a simple loyalty system for a coffee shop using Redis as the backend storage. We'll cover customer management, visit tracking, and temporary discount codes.

## Setup Redis Connection

First, ensure Redis is running using the provided docker-compose.yml:
```bash
docker-compose up -d
```

In [1]:
from redis import Redis
import time

r = Redis(host="localhost", port=6379, decode_responses=True)
r.flushdb()
print("Connected to Redis. Database cleared.")

Connected to Redis. Database cleared.


## Store Customer Information

Redis uses key-value pairs to store data. We'll use the pattern `customer:name` for keys.

In [2]:
r.set("customer:alice", "Alice Johnson")
print("Customer ->", r.get("customer:alice"))

r.set("customer:bob", "Bob Smith")
print("Customer ->", r.get("customer:bob"))

Customer -> Alice Johnson
Customer -> Bob Smith


## Track Customer Visits

Redis INCR command automatically creates keys and increments counters.

In [3]:
print("Alice visits before increment:", r.get("visits:alice"))
print("First visit ->", r.incr("visits:alice"))
print("Second visit ->", r.incr("visits:alice"))
print("Third visit ->", r.incr("visits:alice"))
print("Current visit count:", r.get("visits:alice"))

Alice visits before increment: None
First visit -> 1
Second visit -> 2
Third visit -> 3
Current visit count: 3


## Generate Temporary Discount Codes

Create discount codes with TTL (time-to-live) that automatically expire.

In [4]:
r.set("discount:alice", "SAVE10", ex=10)
print("Discount code ->", r.get("discount:alice"))
print("TTL (seconds remaining) ->", r.ttl("discount:alice"))

time.sleep(3)
print("After 3 seconds:")
print("Discount code ->", r.get("discount:alice"))
print("TTL ->", r.ttl("discount:alice"))

Discount code -> SAVE10
TTL (seconds remaining) -> 10
After 3 seconds:
Discount code -> SAVE10
TTL -> 7


## Customer Lookup Functions

Build helper functions for customer management.

In [5]:
def register_customer(customer_id, full_name):
    key = f"customer:{customer_id}"
    r.set(key, full_name)
    return f"Registered: {full_name}"

def get_customer(customer_id):
    key = f"customer:{customer_id}"
    return r.get(key)

def customer_exists(customer_id):
    key = f"customer:{customer_id}"
    return r.exists(key) == 1

print(register_customer("charlie", "Charlie Brown"))
print("Customer exists:", customer_exists("charlie"))
print("Customer name:", get_customer("charlie"))

Registered: Charlie Brown
Customer exists: True
Customer name: Charlie Brown


## Visit Counter Management

Functions to manage customer visit tracking and rewards.

In [6]:
def record_visit(customer_id):
    key = f"visits:{customer_id}"
    visits = r.incr(key)
    return visits

def get_visit_count(customer_id):
    key = f"visits:{customer_id}"
    count = r.get(key)
    return int(count) if count else 0

def is_eligible_for_reward(customer_id, threshold=5):
    return get_visit_count(customer_id) >= threshold

print("Charlie's visits:", record_visit("charlie"))
print("Charlie's visits:", record_visit("charlie"))
print("Charlie's visits:", record_visit("charlie"))
print("Total visits:", get_visit_count("charlie"))
print("Eligible for reward:", is_eligible_for_reward("charlie", 3))

Charlie's visits: 1
Charlie's visits: 2
Charlie's visits: 3
Total visits: 3
Eligible for reward: True


## Discount Code Validation

Functions to create and validate temporary discount codes.

In [7]:
def generate_discount_code(customer_id, code, expiry_seconds=60):
    key = f"discount:{customer_id}"
    r.set(key, code, ex=expiry_seconds)
    return f"Generated code {code} for {customer_id}, expires in {expiry_seconds}s"

def validate_discount_code(customer_id):
    key = f"discount:{customer_id}"
    code = r.get(key)
    ttl = r.ttl(key)
    
    if code is None:
        return {"valid": False, "message": "No discount code found"}
    if ttl == -2:
        return {"valid": False, "message": "Code has expired"}
    
    return {"valid": True, "code": code, "ttl": ttl}

def redeem_discount_code(customer_id):
    key = f"discount:{customer_id}"
    code = r.get(key)
    if code:
        r.delete(key)
        return f"Redeemed code: {code}"
    return "No valid code to redeem"

print(generate_discount_code("charlie", "SAVE15", 30))
print("Validation:", validate_discount_code("charlie"))
time.sleep(2)
print("After 2 seconds:", validate_discount_code("charlie"))

Generated code SAVE15 for charlie, expires in 30s
Validation: {'valid': True, 'code': 'SAVE15', 'ttl': 30}
After 2 seconds: {'valid': True, 'code': 'SAVE15', 'ttl': 28}


## Complete Loyalty System Demo

Demonstrate the complete workflow with multiple customers.

In [8]:
def complete_customer_workflow(customer_id, full_name):
    print(f"\n--- {full_name} Workflow ---")
    
    if not customer_exists(customer_id):
        print(register_customer(customer_id, full_name))
    
    visits = record_visit(customer_id)
    print(f"Visit recorded. Total visits: {visits}")
    
    if is_eligible_for_reward(customer_id, 2):
        print(generate_discount_code(customer_id, "LOYALTY20", 45))
        validation = validate_discount_code(customer_id)
        print(f"Code validation: {validation}")
    
    return get_visit_count(customer_id)

customers = [
    ("diana", "Diana Prince"),
    ("bruce", "Bruce Wayne"),
    ("clark", "Clark Kent")
]

for customer_id, name in customers:
    complete_customer_workflow(customer_id, name)
    complete_customer_workflow(customer_id, name)

print("\n--- Final Status ---")
for customer_id, name in customers:
    visits = get_visit_count(customer_id)
    validation = validate_discount_code(customer_id)
    print(f"{name}: {visits} visits, Discount: {validation.get('valid', False)}")


--- Diana Prince Workflow ---
Registered: Diana Prince
Visit recorded. Total visits: 1

--- Diana Prince Workflow ---
Visit recorded. Total visits: 2
Generated code LOYALTY20 for diana, expires in 45s
Code validation: {'valid': True, 'code': 'LOYALTY20', 'ttl': 45}

--- Bruce Wayne Workflow ---
Registered: Bruce Wayne
Visit recorded. Total visits: 1

--- Bruce Wayne Workflow ---
Visit recorded. Total visits: 2
Generated code LOYALTY20 for bruce, expires in 45s
Code validation: {'valid': True, 'code': 'LOYALTY20', 'ttl': 45}

--- Clark Kent Workflow ---
Registered: Clark Kent
Visit recorded. Total visits: 1

--- Clark Kent Workflow ---
Visit recorded. Total visits: 2
Generated code LOYALTY20 for clark, expires in 45s
Code validation: {'valid': True, 'code': 'LOYALTY20', 'ttl': 45}

--- Final Status ---
Diana Prince: 2 visits, Discount: True
Bruce Wayne: 2 visits, Discount: True
Clark Kent: 2 visits, Discount: True
